In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.4.1 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_c20k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 350
LEARNING_RATE = 1e-4
CUDA = torch.cuda.is_available()
PATH = 'AG-only-stats-prior-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)
DEVICE = torch.device('cuda:1')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE)

In [3]:
from local_enc import *
from global_enc import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (enc_eta, enc_z)

In [ ]:
from ag_ep import *

train_prior(models, EUBO_init_eta_prior, optimizer, Data, Model_Params, Train_Params)

epoch: 0\350 (152s),  symKL_DB_eta: 308.354,  symKL_DB_z: 433.543,  loss: -1252.872,  ess: 3.005,  kl_eta_ex: 891.020,  kl_eta_in: 150.277,  kl_z_ex: 303.076,  kl_z_in: 37.317
epoch: 1\350 (151s),  symKL_DB_eta: 209.135,  symKL_DB_z: 247.950,  loss: -989.160,  ess: 3.354,  kl_eta_ex: 527.947,  kl_eta_in: 130.426,  kl_z_ex: 163.052,  kl_z_in: 27.892
epoch: 2\350 (150s),  symKL_DB_eta: 149.661,  symKL_DB_z: 149.874,  loss: -858.273,  ess: 3.670,  kl_eta_ex: 338.887,  kl_eta_in: 120.987,  kl_z_ex: 90.196,  kl_z_in: 20.998
epoch: 3\350 (146s),  symKL_DB_eta: 110.861,  symKL_DB_z: 100.895,  loss: -781.330,  ess: 3.969,  kl_eta_ex: 237.975,  kl_eta_in: 120.578,  kl_z_ex: 55.606,  kl_z_in: 16.084
epoch: 4\350 (151s),  symKL_DB_eta: 87.755,  symKL_DB_z: 74.379,  loss: -732.990,  ess: 4.239,  kl_eta_ex: 183.393,  kl_eta_in: 131.037,  kl_z_ex: 38.556,  kl_z_in: 12.692
epoch: 5\350 (151s),  symKL_DB_eta: 75.029,  symKL_DB_z: 60.644,  loss: -704.494,  ess: 4.438,  kl_eta_ex: 155.413,  kl_eta_in: 1

In [ ]:
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)
torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-%s" % PATH)